## Pair of Teams Past Matchup Data

In [38]:
import logging
from pathlib import Path
import datetime
log_file = Path(f"log/log.log")
log_file.parent.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="w"  # Overwrite the file each time
)
logger = logging.getLogger(__name__)


In [39]:
from itertools import combinations
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import SeasonType
import pandas as pd
from nba_api.stats.static import teams
from pathlib import Path

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
TEAM_IDS = {team['abbreviation']: team['id'] for team in nba_teams}    

In [40]:
from rich.table import Table
from rich.jupyter import print

def show_summary_table(games_summary):
    
    # Create a Rich Table
    table = Table(title="2024-2025 Regular Season Matchup Records")
    table.add_column("Team A", justify="left")
    table.add_column("Team B", justify="left")
    table.add_column("Point Difference", justify="right")
    table.add_column("Record", justify="center")
    
    # Add rows to the table
    for teams, info in games_summary.items():
        a, b = teams
        d = int(info['diff_pts'])
        r = info['record']
        table.add_row(a, b, str(d), r)
    
    # Print the table
    print(table)



In [78]:
from itertools import combinations
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import SeasonType
import pandas as pd

def get_all_matchups(team_abbreviations):
    """
    Fetches all matchups for the given team abbreviations and returns a DataFrame.

    Parameters:
        team_abbreviations (set): A set of team abbreviations (e.g., {'BOS', 'TOR'}).

    Returns:
        pd.DataFrame: A DataFrame with columns ['date', 'team', 'opp_team', 'pts', 'opp_team_pts'].
    """
    matchups = []
    for team_abbr, vs_team_abbr in combinations(team_abbreviations, 2):
        team_id, vs_team_id = TEAM_IDS[team_abbr], TEAM_IDS[vs_team_abbr]
        gamefinder = leaguegamefinder.LeagueGameFinder(
            team_id_nullable=team_id,
            vs_team_id_nullable=vs_team_id,
            season_nullable="2024-25",
            season_type_nullable=SeasonType.regular
        )
        games = gamefinder.get_data_frames()[0]
        logger.info(games[['GAME_DATE', 'MATCHUP', 'PTS', 'PLUS_MINUS']])
        for _, game in games.iterrows():
            pts_opp = int(game['PTS'] - game['PLUS_MINUS'])
            matchup = {
                'date': game['GAME_DATE'],
                'team': game['MATCHUP'].split(' ')[0],
                'opp_team': game['MATCHUP'].split(' ')[-1],
                'pts': game['PTS'],
                'opp_team_pts': pts_opp
            }
            matchups.append(matchup)
    frame = pd.DataFrame(matchups)
    logger.info(frame[['date', 'team', 'opp_team', 'pts', 'opp_team_pts']])
    return frame

def get_all_matchups_summary(team_abbreviations):
    """
    Calculates the total point differences for each pair of teams in the most current regular season.

    Parameters:
        team_abbreviations (set): A set of team abbreviations (e.g., {'BOS', 'TOR'}).

    Returns:
        dict: A dictionary where keys are 2-tuples of team abbreviations, and values are the total difference in points scored by the two teams
        over all games where they played each other in the most recent regular season.
    """
    # Fetch all matchups
    matchups_df = get_all_matchups(team_abbreviations)

    # Initialize a dictionary to store point differences
    point_differences = {}
    for team_abbr, vs_team_abbr in combinations(team_abbreviations, 2):
        info = {}
        games = matchups_df[
            ((matchups_df['team'] == team_abbr) & (matchups_df['opp_team'] == vs_team_abbr)) |
            ((matchups_df['team'] == vs_team_abbr) & (matchups_df['opp_team'] == team_abbr))
        ]
        info['#'] = len(games)
        game_diffs = games['pts'] - games['opp_team_pts']
        w = len([x for x in game_diffs if x > 0])
        l = len(game_diffs) - w
        info['diff_pts'] = game_diffs.sum()
        info['record'] = f"{w}-{l}"
        info['total_pts'] = games['pts'].sum() + games['opp_team_pts'].sum()
        point_differences[(team_abbr, vs_team_abbr)] = info
        logger.info(games[['date', 'team', 'opp_team', 'pts', 'opp_team_pts']].to_string(index=False))
        

    return point_differences

def summarize_matchups(matchups_df):
    # Initialize a dictionary to store point differences
    point_differences = {}
    for _, game in matchups_df.iterrows():
        info = point_differences.setdefault((game['team'], game['opp_team']), {})
        info['#'] = info.setdefault('#', 0) + 1
        this_game_diff = game['pts'] - game['opp_team_pts']
        info['diff_pts'] = info.setdefault('diff_pts', 0) + this_game_diff
        w, l = info.setdefault('record', '0-0',).split('-')
        w, l = int(w), int(l)
        if this_game_diff == 0:
            raise "A TIE GAME"
        w, l = w + int(this_game_diff > 0), l + int(this_game_diff < 0)
        info['record'] = f"{w}-{l}"
        info['total_pts'] = info.setdefault('total_pts', 0) + game['pts']
        logger.info(info)

    return point_differences

In [ ]:
def get_all_matchups_summary_new(team_abbreviations):
    """
    Calculates the total point differences for each pair of teams in the most current regular season.

    Parameters:
        team_abbreviations (set): A set of team abbreviations (e.g., {'BOS', 'TOR'}).

    Returns:
        dict: A dictionary where keys are 2-tuples of team abbreviations, and values are the total difference in points scored by the two teams
        over all games where they played each other in the most recent regular season.
    """
    # Fetch all matchups
    matchups_df = get_all_matchups(team_abbreviations)

    return summarize_matchups(matchups_df)

In [79]:
west = {'OKC', 'DEN', 'MIN', 'GSW'}
east = {'IND', 'BOS', 'NYK', 'CLE'}
matchups_df = get_all_matchups(east)
print(matchups_df)
show_summary_table(summarize_matchups(matchups_df))

NEW

4

GAME_DATE      MATCHUP  PTS  PLUS_MINUS
0  2025-04-11  NYK vs. CLE  102        -6.0
1  2025-04-02    NYK @ CLE  105       -19.0
2  2025-02-21    NYK @ CLE  105       -37.0
3  2024-10-28  NYK vs. CLE  104        -6.0

4

GAME_DATE      MATCHUP  PTS  PLUS_MINUS
0  2025-04-08  NYK vs. BOS  117        -2.0
1  2025-02-23    NYK @ BOS  105       -13.0
2  2025-02-08  NYK vs. BOS  104       -27.0
3  2024-10-22    NYK @ BOS  109       -23.0

...

3

GAME_DATE      MATCHUP  PTS  PLUS_MINUS
0  2024-12-29  BOS vs. IND  114        -9.0
1  2024-12-27  BOS vs. IND  142        37.0
2  2024-10-30    BOS @ IND  132        -3.0

date team opp_team  pts  opp_team_pts
0   2025-04-11  NYK      NYK  102           108
1   2025-04-02  NYK      CLE  105           124
2   2025-02-21  NYK      CLE  105           142
3   2024-10-28  NYK      NYK  104           110
4   2025-04-08  NYK      NYK  117           119
5   2025-02-23  NYK      BOS  105           118
6   2025-02-08  NYK      NYK  104           131
7   2024-10-22  NYK      BOS  109           132
8   2025-02-11  NYK      IND  128           115
9   2024-11-10  NYK      IND  121           132
10  2024-10-25  NYK      NYK  123            98
11  2025-02-28  CLE      BOS  123           116
12  2025-02-04  CLE      CLE  105           112
13  2024-12-01  CLE      CLE  115           111
14  2024-11-19  CLE      BOS  117           120
15  2025-04-13  CLE      CLE  118           126
16  2025-04-10  CLE      IND  112           114
17  2025-01-14  CLE      IND  127           117
18  2025-01-12  CLE      CLE   93           108
19  2024-12-29  BOS      BOS  114           123
20  2024-12-27  BOS      BOS  142           105
21  2024-10-30  BOS      IND  132           135

   2024-2025 Regular Season Matchup Records    
┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Team A ┃ Team B ┃ Point Difference ┃ Record ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ NYK    │ CLE    │              -56 │  0-2   │
│ NYK    │ BOS    │              -36 │  0-2   │
│ NYK    │ IND    │                2 │  1-1   │
│ CLE    │ BOS    │                4 │  1-1   │
│ CLE    │ IND    │                8 │  1-1   │
│ BOS    │ IND    │               -3 │  0-1   │
└────────┴────────┴──────────────────┴────────┘

   2024-2025 Regular Season Matchup Records    
┏━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Team A ┃ Team B ┃ Point Difference ┃ Record ┃
┡━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ NYK    │ CLE    │              -68 │  0-4   │
│ NYK    │ BOS    │              -65 │  0-4   │
│ NYK    │ IND    │               27 │  2-1   │
│ CLE    │ BOS    │                1 │  2-2   │
│ CLE    │ IND    │              -15 │  1-3   │
│ BOS    │ IND    │               25 │  1-2   │
└────────┴────────┴──────────────────┴────────┘